In [5]:
import pandas as pd
import math, time
from numpy import save,load

#### Display a number with commas 

In [14]:
import re

def comma_me(amount):
    if type(amount) == int:
        amount = str(amount)
    orig = amount
    new = re.sub("^(-?\d+)(\d{3})", '\g<1>,\g<2>', amount)
    if orig == new:
        return new
    else:
        return comma_me(new)

#### Slicing the first and last n columns in a dataframe

In [15]:
def front(self, n):
    return self.iloc[:, :n]

def back(self, n):
    return self.iloc[:, -n:]

pd.DataFrame.front = front
pd.DataFrame.back = back

#### Method to load and save an object as numpy

In [6]:
def loadNumpy(name,path='.'):
    if ".npy" in name:
        fullPath = path+'/'+name
    else:
        fullPath = path+'/'+name+'.npy'
    return load(fullPath)


def saveNumpy(obj, name, path='.'):
    if ".npy" not in name:
        fullPath = path+'/'+name
        save(fullPath, obj)
        print name,'saved successfully in',path
    else:
        fullPath = path+'/'+name.split(".npy")[0]
        save(fullPath, obj)
        print name,'saved successfully in',path
        
def loadDict(name,path='.'):
    if ".npy" in name:
        fullPath = path+'/'+name
    else:
        fullPath = path+'/'+name+'.npy'
    return load(fullPath).tolist()

def saveDict(obj, name, path='.'):
    if ".dict" not in name:
        fullPath = path+'/'+name+'.dict'
        save(fullPath, obj)
        print name,'saved successfully in',path
    else:
        fullPath = path+'/'+name
        save(fullPath, obj)
        print name,'saved successfully in',path

In [17]:
DEFAULT_TEMP = 300

#### Method to read the toolpath from file

In [18]:
def parse_toolpath(FolderName,FileName,dt):
    toolpath_raw=pd.read_table(FolderName+FileName,delimiter=r"\s+",header=None, names=['time','x','y','z','state'])
    toolpath=[]
    state=[]
    time=0.0
    ind=0
    endTime = float(toolpath_raw.tail(1)['time'])
    while(time<=endTime):
        while(time>=toolpath_raw['time'][ind+1]):
            ind=ind+1
        X=toolpath_raw['x'][ind]+(toolpath_raw['x'][ind+1]-toolpath_raw['x'][ind])*(
            time-toolpath_raw['time'][ind])/(toolpath_raw['time'][ind+1]-toolpath_raw['time'][ind])
        Y=toolpath_raw['y'][ind]+(toolpath_raw['y'][ind+1]-toolpath_raw['y'][ind])*(
            time-toolpath_raw['time'][ind])/(toolpath_raw['time'][ind+1]-toolpath_raw['time'][ind])
        Z=toolpath_raw['z'][ind]+(toolpath_raw['z'][ind+1]-toolpath_raw['z'][ind])*(
            time-toolpath_raw['time'][ind])/(toolpath_raw['time'][ind+1]-toolpath_raw['time'][ind])
        toolpath.append([X,Y,Z])
        state.append(toolpath_raw['state'][ind+1])
        time = time +dt
    return toolpath, state, endTime

In [7]:
toolpath, state, endTime = parse_toolpath('data/','toolpath.crs',0.1)

In [8]:
len(toolpath)

2522

In [9]:
toolpath[:10]

[[0.0, 0.75, 0.5],
 [2.5999999974000003, 0.75, 0.5],
 [5.199999994800001, 0.75, 0.5],
 [7.799999992200002, 0.75, 0.5],
 [10.399999989600001, 0.75, 0.5],
 [12.999999987, 0.75, 0.5],
 [15.5999999844, 0.75, 0.5],
 [18.1999999818, 0.75, 0.5],
 [20.0, 1.5499999480000004, 0.5],
 [18.100000081900003, 2.25, 0.5]]

#### Creates a point on a coordinate plane with values x, y and z.

In [10]:
class Coordinate(object):

    def __init__(self, x,y,z ):
        self.X = x
        self.Y = y
        self.Z = z
#         print type(tupleObject)
#         self.X = tupleObject[0]
#         self.Y = tupleObject[1]
#         self.Z = tupleObject[2]

    def __str__(self):
        return "Coordinate(%s,%s,%s)"%(self.X, self.Y, self.Z) 

    def getX(self):
        return self.X

    def getY(self):
        return self.Y
    
    def getZ(self):
        return self.Z
    
    def getXYZ(self):
        return self.X, self.Y, self.Z

    def distance(self, other):
        dx = self.X - other.X
        dy = self.Y - other.Y
        dz = self.Z - other.Z
        return math.sqrt(dx**2 + dy**2 + dz**2)

### Describe a point that is a coordinate with also t axis 

In [11]:
class Point(object):

    COUNT = 0

    def __init__(self, tupleObject ):
        '''Defines x and y variables'''
        self.X = tupleObject[0]
        self.Y = tupleObject[1]
        self.Z = tupleObject[2]
        self.t = tupleObject[3]

    def __str__(self):
        return "Point(%s,%s,%s,%s)"%(self.X, self.Y, self.Z, self.t) 

    def getX(self):
        return self.X

    def getY(self):
        return self.Y
    
    def getZ(self):
        return self.Z
    
    def getTime(self):
        return self.t

    def distance(self, other):
        dx = self.X - other.X
        dy = self.Y - other.Y
        dz = self.Z - other.Z
        return math.sqrt(dx**2 + dy**2 + dz**2)

In [12]:
print Point((12.5, 5.0, 0.5, 2.1))

Point(12.5,5.0,0.5,2.1)


#### Read the csv file

In [13]:
df = pd.read_csv('data/20-20-10-800.csv',header=None)

#### Transpose the dataframe so that rows indicate different voxels and columns indicate temperature at different time steps

In [15]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,33611,33612,33613,33614,33615,33616,33617,33618,33619,33620
0,0.500,0.500,0.000,0.500,0.000,0.000,0.000,0.500,1.0000,1.0000,...,4.000,3.500,3.000,2.500,2.00,1.500,1.000,0.500,0.000,
1,1.500,1.000,0.500,0.500,0.000,1.000,1.500,0.000,0.5000,1.0000,...,20.000,20.000,20.000,20.000,20.00,20.000,20.000,20.000,20.000,
2,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.500,0.5000,0.5000,...,10.000,10.000,10.000,10.000,10.00,10.000,10.000,10.000,10.000,
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0077,0.0077,...,251.973,251.988,252.012,252.027,252.05,252.065,252.089,252.104,252.127,
4,300.000,300.000,300.000,300.000,300.000,300.000,300.000,300.000,300.0000,300.0000,...,300.000,300.000,300.000,300.000,300.00,300.000,300.000,300.000,300.000,
5,1876.720,1959.090,1643.900,1845.840,1554.230,1763.850,1746.630,1754.330,2706.9800,2869.6700,...,300.000,300.000,300.000,300.000,300.00,300.000,300.000,300.000,300.000,
6,1069.290,1068.080,1019.100,1067.810,1015.720,1020.860,1017.880,1065.690,1209.5300,1210.2400,...,300.000,300.000,300.000,300.000,300.00,300.000,300.000,300.000,300.000,
7,895.958,896.858,864.489,897.931,863.362,863.354,861.520,897.344,991.5430,991.0750,...,300.000,300.000,300.000,300.000,300.00,300.000,300.000,300.000,300.000,
8,786.233,787.094,762.822,788.166,762.239,761.559,760.245,787.972,859.2210,858.6720,...,300.000,300.000,300.000,300.000,300.00,300.000,300.000,300.000,300.000,
9,706.877,707.581,688.495,708.430,688.116,687.467,686.517,708.326,764.3710,763.9100,...,300.000,300.000,300.000,300.000,300.00,300.000,300.000,300.000,300.000,


In [16]:
len(df)

2526

In [17]:
dataframe = df.transpose()

In [13]:
saveNumpy(dataframe,'whole_Data')

whole_Data saved successfully in .


In [37]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525
0,0.5,1.5,0.5,0,300,1876.72,1069.29,895.958,786.233,706.877,...,1131.88,1131.61,1131.32,1131.03,1130.73,1130.43,1130.11,1129.78,1129.45,1129.11
1,0.5,1,0.5,0,300,1959.09,1068.08,896.858,787.094,707.581,...,1121.15,1120.88,1120.61,1120.33,1120.04,1119.74,1119.44,1119.13,1118.8,1118.47
2,0,0.5,0.5,0,300,1643.9,1019.1,864.489,762.822,688.495,...,1104.93,1104.68,1104.43,1104.16,1103.89,1103.62,1103.33,1103.04,1102.74,1102.43
3,0.5,0.5,0.5,0,300,1845.84,1067.81,897.931,788.166,708.43,...,1109.71,1109.46,1109.2,1108.93,1108.65,1108.37,1108.08,1107.78,1107.47,1107.16
4,0,0,0.5,0,300,1554.23,1015.72,863.362,762.239,688.116,...,1100,1099.76,1099.51,1099.26,1098.99,1098.72,1098.44,1098.15,1097.86,1097.56


### <font color="magenta"> No. of voxels</font>

In [14]:
len(dataframe)

33621

In [ ]:
len(dataframe)

### <font color='bieg'>No. of time steps </font>

In [ ]:
len(dataframe.loc[0])-4

### <font color="orange"> No. of possible rows = No. of voxels x  No. of time steps ~ 80 million </font>

In [219]:
comma_me(len(dataframe)*len(dataframe.loc[0]-4))

'80,186,085'

In [ ]:
400 * 3 * 340

#### <font color="red">If we reduce no. of layers by 7 times , we reduce no. of time steps also by 7 times. So we approximately need only 1.6 million data points </font>

* No. of time steps = 340 
* No. of layers = 3
* No. of voxels = 4803

In [32]:
dfRelevant = dataframe.front(5)

In [26]:
smallSlice = dataframe[:4803].front(340)

In [44]:
bigSlice = dataframe[:15000].front(1200)

In [45]:
bigSlice.tail()

,0,1,2,3,4,5,6,7,8,9,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
14995,5,18.5,4.5,111.403,300,300,300,300,300,300,...,1088.32,1084.11,1080.09,1076.04,1072.39,1068.92,1065.63,1062.53,1059.67,1057.09
14996,5,19,4.5,111.403,300,300,300,300,300,300,...,1085.7,1081.37,1077.24,1073.06,1069.3,1065.71,1062.3,1059.07,1056.04,1053.24
14997,5,19.5,4.5,111.41,300,300,300,300,300,300,...,1083.61,1079.22,1075.02,1070.77,1066.94,1063.29,1059.81,1056.51,1053.39,1050.47
14998,5.5,18.5,4.5,111.418,300,300,300,300,300,300,...,1092.27,1088.01,1083.95,1079.84,1076.16,1072.65,1069.33,1066.21,1063.31,1060.69
14999,5.5,19,4.5,111.418,300,300,300,300,300,300,...,1089.65,1085.27,1081.09,1076.87,1073.06,1069.43,1065.99,1062.73,1059.66,1056.83


In [42]:
bigSlice.head()

,0,1,2,3,4,5,6,7,8,9,...,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019
0,0.5,1.5,0.5,0,300,1876.72,1069.29,895.958,786.233,706.877,...,924.829,923.039,921.268,919.515,917.781,916.064,914.366,912.685,911.11,916.237
1,0.5,1,0.5,0,300,1959.09,1068.08,896.858,787.094,707.581,...,914.341,912.633,910.94,909.265,907.607,905.965,904.34,902.732,901.239,907.364
2,0,0.5,0.5,0,300,1643.9,1019.1,864.489,762.822,688.495,...,900.219,898.616,897.028,895.455,893.898,892.355,890.828,889.315,887.926,894.818
3,0.5,0.5,0.5,0,300,1845.84,1067.81,897.931,788.166,708.43,...,904.08,902.45,900.835,899.236,897.653,896.085,894.532,892.995,891.578,898.32
4,0,0,0.5,0,300,1554.23,1015.72,863.362,762.239,688.116,...,896.053,894.48,892.921,891.377,889.848,888.333,886.833,885.348,883.986,891.082


In [27]:
smallSlice.tail()

,0,1,2,3,4,5,6,7,8,9,...,330,331,332,333,334,335,336,337,338,339
4798,13,17.5,1.5,34.6642,300,300,300,300,300,300,...,300,300,300,300,300,300,300,300,300,300
4799,13,17,1.5,34.6642,300,300,300,300,300,300,...,300,300,300,300,300,300,300,300,300,300
4800,13,18,1.5,34.6719,300,300,300,300,300,300,...,300,300,300,300,300,300,300,300,300,300
4801,12.5,17.5,1.5,34.6796,300,300,300,300,300,300,...,300,300,300,300,300,300,300,300,300,300
4802,12.5,17,1.5,34.6796,300,300,300,300,300,300,...,300,300,300,300,300,300,300,300,300,300


#### Method to find the positions of neighbors (in terms of Coordinate object)

In [38]:
def findNeighbors(p):
    
    neighbors = []
    ### immediate/adjacent neighbors - 1 degree 
    neighbors+= [Coordinate(p.X - 0.5, p.Y, p.Z)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y, p.Z)]
    
    neighbors+= [Coordinate(p.X, p.Y - 0.5, p.Z)]
    neighbors+= [Coordinate(p.X, p.Y + 0.5, p.Z)]
    
    neighbors+= [Coordinate(p.X, p.Y, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X, p.Y, p.Z + 0.5)]
    
    ### distance - 2 degree
    neighbors+= [Coordinate(p.X - 0.5, p.Y - 0.5, p.Z)]
    neighbors+= [Coordinate(p.X - 0.5, p.Y + 0.5, p.Z)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y - 0.5, p.Z)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y + 0.5, p.Z)]
    
    neighbors+= [Coordinate(p.X, p.Y - 0.5, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X, p.Y + 0.5, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X, p.Y - 0.5, p.Z + 0.5)]
    neighbors+= [Coordinate(p.X, p.Y + 0.5, p.Z + 0.5)]
    
    neighbors+= [Coordinate(p.X - 0.5, p.Y, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X - 0.5, p.Y, p.Z + 0.5)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y, p.Z + 0.5)]
    
    ### diagonal neighbors distance - 3 degree
    neighbors+= [Coordinate(p.X - 0.5, p.Y - 0.5, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X - 0.5, p.Y + 0.5, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y - 0.5, p.Z - 0.5)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y + 0.5, p.Z - 0.5)]
    
    neighbors+= [Coordinate(p.X - 0.5, p.Y - 0.5, p.Z + 0.5)]
    neighbors+= [Coordinate(p.X - 0.5, p.Y + 0.5, p.Z + 0.5)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y - 0.5, p.Z + 0.5)]
    neighbors+= [Coordinate(p.X + 0.5, p.Y + 0.5, p.Z + 0.5)]
    
    return neighbors

#### Method to find the positions of neighbors (in terms of indices in original dataframe)

In [39]:
def findNeighborIndices(p):
    
    neighbors = []
    
    
    ### immediate/adjacent neighbors - 1 degree 
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y, p.Z))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y, p.Z))]
    
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y - 0.5, p.Z))]
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y + 0.5, p.Z))]
    
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y, p.Z + 0.5))]
    
    ### distance - 2 degree
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z))]
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z))]
    
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y - 0.5, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y + 0.5, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y - 0.5, p.Z + 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X, p.Y + 0.5, p.Z + 0.5))]
    
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y, p.Z + 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y, p.Z + 0.5))]
    
    ### diagonal neighbors distance - 3 degree
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z - 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z - 0.5))]
    
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z + 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z + 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z + 0.5))]
    neighbors+= [getLocationIndices(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z + 0.5))]
    
    return neighbors

#### Method to find the temperature of neighbors at first time step

In [40]:
def findNeighborTempsInit(p):
    
    neighbors = []
    
    ### immediate/adjacent neighbors - 1 degree 
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y, p.Z))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y, p.Z))]
    
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y - 0.5, p.Z))]
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y + 0.5, p.Z))]
    
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y, p.Z + 0.5))]
    
    ### distance - 2 degree
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z))]
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z))]
    
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y - 0.5, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y + 0.5, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y - 0.5, p.Z + 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X, p.Y + 0.5, p.Z + 0.5))]
    
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y, p.Z + 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y, p.Z + 0.5))]
    
    ### diagonal neighbors distance - 3 degree
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z - 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z - 0.5))]
    
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z + 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z + 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z + 0.5))]
    neighbors+= [getTemperatureInit(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z + 0.5))]
    
    return neighbors

#### Method to find the temperatures of neighbors for this timestep

In [41]:
def findNeighborTemperatures(p,timestep=1):
    
    neighbors = []
    
    ### immediate/adjacent neighbors - 1 degree 
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y, p.Z),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y, p.Z),timestep)]
    
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y - 0.5, p.Z),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y + 0.5, p.Z),timestep)]
    
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y, p.Z + 0.5),timestep)]
    
    ### distance - 2 degree
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z),timestep)]
    
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y - 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y + 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y - 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X, p.Y + 0.5, p.Z + 0.5),timestep)]
    
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y, p.Z + 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y, p.Z + 0.5),timestep)]
    
    ### diagonal neighbors distance - 3 degree
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z - 0.5),timestep)]
    
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y - 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X - 0.5, p.Y + 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y - 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getTemperatures(Coordinate(p.X + 0.5, p.Y + 0.5, p.Z + 0.5),timestep)]
    
    return neighbors        

### <font color="inkblue">Method to find the temperatures of neighbors  </font>
#### timestep adjusted based on if the temperature for this timestep is available

In [42]:
def findAdjustedNeighborTemperatures(p,timestep=1):
    
    neighbors = []
#     start = time.time()
    
    ### immediate/adjacent neighbors - 1 degree 
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y, p.Z),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y, p.Z),timestep)]
    
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y - 0.5, p.Z),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y + 0.5, p.Z),timestep)]
    
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y, p.Z + 0.5),timestep)]
    
    ### distance - 2 degree
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y - 0.5, p.Z),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y + 0.5, p.Z),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y - 0.5, p.Z),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y + 0.5, p.Z),timestep)]
    
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y - 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y + 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y - 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X, p.Y + 0.5, p.Z + 0.5),timestep)]
    
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y, p.Z + 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y, p.Z + 0.5),timestep)]
    
    ### diagonal neighbors distance - 3 degree
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y - 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y + 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y - 0.5, p.Z - 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y + 0.5, p.Z - 0.5),timestep)]
    
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y - 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X - 0.5, p.Y + 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y - 0.5, p.Z + 0.5),timestep)]
    neighbors+= [getAdjustedTemperatures(p,Coordinate(p.X + 0.5, p.Y + 0.5, p.Z + 0.5),timestep)]
#     stop = time.time()
#     print "The time elapse is",stop-start," seconds"
    return neighbors

In [32]:
indices = loadNumpy('indices_slice').tolist()
p= Coordinate(0.5,1.0,0.5)


300

In [59]:
p= Coordinate(13.0, 13.5, 1.0)
xyzTuple = p.getXYZ()
timestep = 250
temp_timestep =-1
if xyzTuple in indices.keys() and timestep>0:
    dfIndex = indices[xyzTuple]
    temp_timestep = dataframe.loc[dfIndex].tolist()[timestep+4]

In [61]:
getTemperature(p,250)

783.453

In [43]:
def printNeighbors(p):
    neighbors = findNeighbors(p)
    print "The neighbors of point",p," are:"
    print
    print "Immediate Neighbors: "
    print 
    for i in range(0,6):
        print neighbors[i]
    
    print
    print "2-D diagonal Neighbors: "
    print 
    for i in range(6,18):
        print neighbors[i]
    
    print
    print "3-D diagonal Neighbors: "
    print 
    for i in range(18,26):
        print neighbors[i]

In [44]:
def getLocationIndices(p):
       
    xyzTuple = p.getXYZ()
    if xyzTuple in indices.keys():
        return indices[xyzTuple]
    else:
        return -1

In [45]:
def getTemperatureInit(p):
    
    xyzTuple = p.getXYZ()
    if xyzTuple in indices.keys():
        return valueDict[xyzTuple][1]
    else:
        return DEFAULT_TEMP


In [46]:
def getTemperature(p,timestep):
    xyzTuple = p.getXYZ()
    if xyzTuple in indices.keys() and timestep>0:
        dfIndex = indices[xyzTuple]
        temp_timestep = dataframe.loc[dfIndex].tolist()[timestep+4]
        return temp_timestep
    else:
        return DEFAULT_TEMP

In [47]:
def getAdjustedTemperature(self,other,timestep):
    selfTuple = self.getXYZ()
    otherTuple = other.getXYZ()
    
    if selfTuple in indices.keys() and otherTuple in indices.keys() and timestep>0:
        index_self = indices[selfTuple]
        index_other = indices[otherTuple]
        if index_other<index_self:
            temp_timestep = dataframe.loc[index_other].tolist()[timestep+4]
            return temp_timestep
        else:
            if timestep>1:
                temp_timestep = dataframe.loc[index_other].tolist()[timestep-1+4]
                return temp_timestep
            
    return DEFAULT_TEMP

In [289]:
def createDatabase(dataframe=dataframe):
    neighborColumns = []
    
    for neighbor in range(26):
        neighborColumns += ['T'+str(neighbor+1)+'(t/t-1)']
    
    columns = ['timestep','x(voxel)','y(voxel)','z(voxel)','timeOfCreation','x(laser)','y(laser)','z(laser)','distance(laser)','T0(t-1)'] + neighborColumns
    database = pd.DataFrame(columns=columns)


    indices = loadNumpy('indices_slice')
    
    num_timesteps = len(dataframe.loc[0]-4)
    num_voxels = len(dataframe)
    
    for timestep in range(1,5):#range(1,num_timesteps+1):
        for voxel in indices.keys():
            x,y,z = voxel
            x_laser,y_laser,z_laser = tuple(toolpath[timestep-1])

            voxelPosObject = Coordinate(x,y,z)
            laserPosObject = Coordinate(x_laser,y_laser,z_laser)
            distance_laser = voxelPosObject.distance(laserPosObject)
            
            dfIndex = indices[voxel]
            row = dataframe.loc[dfIndex].tolist()
            time_creation = row[3]
            
            T_voxel_previous = getTemperature(voxelPosObject,timestep-1)
            
            dictionary = {'timestep':timestep, 'x(voxel)':x, 'y(voxel)':y, 'z(voxel)':z, 
                         'timeOfCreation':time_creation, 'x(laser)':x_laser,'y(laser)':y_laser,'z(laser)':z_laser,
                          'distance(laser)':distance_laser, 'T0(t-1)':T_voxel_previous}
            
            neighborTemps = findAdjustedNeighborTemperatures(voxelPosObject,timestep)
            for i in range(len(neighborTemps)):
                key = 'T'+str(i+1)+'(t/t-1)'
                dictionary.append({key:neighborTemps[i]})
            database.append([dictionary])

    return database
            
            
createDatabase()

AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [ ]:
indices = loadNumpy('indices_slice')
indices

In [286]:
test = {1:2,3:4}
test.update({4:5,5:6})
test

{1: 2, 3: 4, 4: 5, 5: 6}

In [281]:
Coordinate(test)

TypeError: __init__() takes exactly 4 arguments (2 given)

In [30]:
tinySlice = smallSlice.front(10)[:20]

In [31]:
tinySlice

,0,1,2,3,4,5,6,7,8,9
0,0.5,1.5,0.5,0,300,1876.72,1069.29,895.958,786.233,706.877
1,0.5,1,0.5,0,300,1959.09,1068.08,896.858,787.094,707.581
2,0,0.5,0.5,0,300,1643.9,1019.1,864.489,762.822,688.495
3,0.5,0.5,0.5,0,300,1845.84,1067.81,897.931,788.166,708.43
4,0,0,0.5,0,300,1554.23,1015.72,863.362,762.239,688.116
5,0,1,0.5,0,300,1763.85,1020.86,863.354,761.559,687.467
6,0,1.5,0.5,0,300,1746.63,1017.88,861.52,760.245,686.517
7,0.5,0,0.5,0,300,1754.33,1065.69,897.344,787.972,708.326
8,1,0.5,0.5,0.0077,300,2706.98,1209.53,991.543,859.221,764.371
9,1,1,0.5,0.0077,300,2869.67,1210.24,991.075,858.672,763.91


In [11]:
def createDictionary(dataframe):
    start = time.time()
    dictionary = {}
    for i in range(len(dataframe)):
        key = tuple(dataframe.loc[i][:4].tolist())
        value = dataframe.loc[i][4:].tolist()
        dictionary[key] = value
    stop = time.time()
    print 'The time for dictionary creation is %2f seconds' %(stop-start)
    return dictionary
    

In [10]:
def createObjectDictionaries(dataframe):
    start = time.time()
    dictionaryIndices,dictionaryValues = {},{}
    for i in range(len(dataframe)):
        (x,y,z) = dataframe.loc[i][:3].tolist()
        key = (x,y,z)
        value = dataframe.loc[i][3:].tolist()
        dictionaryIndices[key] = i
        dictionaryValues[key] = value
    stop = time.time()
    print 'The time for dictionary creation is %2f seconds' %(stop-start)
    return dictionaryIndices,dictionaryValues

In [51]:
indices_biggerSlice,value_biggerSlice = createObjectDictionaries(biggerSlice)

The time for dictionary creation is 8.055754 seconds


In [26]:
indices_multi,value_multi = createObjectDictionaries(dataframe)

The time for dictionary creation is 25.535220 seconds


In [48]:
indices_big,valueDict_big = createObjectDictionaries(bigSlice)

The time for dictionary creation is 14.213606 seconds


In [49]:
len(indices_big)

15000

In [50]:
bigSlice.to_csv('data_big.csv')

In [51]:
saveDict(indices_big,'indices_big')

indices_big saved successfully in .


In [175]:
saveNumpy(indices,'indicesDict')
saveNumpy(valueDict,'valueDict')

indicesDict saved successfully in .
valueDict saved successfully in .


In [35]:
indices = loadDict('indices')

In [197]:
saveNumpy(dataframe,'data')

data saved successfully in .


In [222]:
indices,valueDict = createObjectDictionaries(smallSlice)
saveNumpy(indices,'indices_slice')
saveNumpy(valueDict,'values_slice')
saveNumpy(smallSlice,'sliced_dataframe')

The time for dictionary creation is 2.807802 seconds
indices_slice saved successfully in .
values_slice saved successfully in .
sliced_dataframe saved successfully in .


In [154]:
indices.values()[:20], len(indices)

([33498,
  27127,
  27126,
  27128,
  13,
  27129,
  33537,
  27130,
  32817,
  27131,
  27132,
  27133,
  27134,
  27135,
  27136,
  32818,
  27137,
  112,
  27138,
  27139],
 33621)

In [132]:
print smallDict.keys()[1]
testPoint = smallDict.keys()[1]

Point(0.5,1.0,0.5)


In [282]:
x= Coordinate(0.5,1.0,0.5)
y = Coordinate(0.0,1.0,0.0)
x.distance(y)

0.7071067811865476

In [149]:
printNeighbors(Coordinate(0.0,1.0,0.0))

The neighbors of point Point(0.0,1.0,0.0)  are:

Immediate Neighbors: 

Point(-0.5,1.0,0.0)
Point(0.5,1.0,0.0)
Point(0.0,0.5,0.0)
Point(0.0,1.5,0.0)
Point(0.0,1.0,-0.5)
Point(0.0,1.0,0.5)

2-D diagonal Neighbors: 

Point(-0.5,0.5,0.0)
Point(-0.5,1.5,0.0)
Point(0.5,0.5,0.0)
Point(0.5,1.5,0.0)
Point(0.0,0.5,-0.5)
Point(0.0,1.5,-0.5)
Point(0.0,0.5,0.5)
Point(0.0,1.5,0.5)
Point(-0.5,1.0,-0.5)
Point(0.5,1.0,-0.5)
Point(-0.5,1.0,0.5)
Point(0.5,1.0,0.5)

3-D diagonal Neighbors: 

Point(-0.5,0.5,-0.5)
Point(-0.5,1.5,-0.5)
Point(0.5,0.5,-0.5)
Point(0.5,1.5,-0.5)
Point(-0.5,0.5,0.5)
Point(-0.5,1.5,0.5)
Point(0.5,0.5,0.5)
Point(0.5,1.5,0.5)


### <font color="green">Test Location and Temperature Extraction Functions </font>

In [204]:
findNeighborIndices(Coordinate(16.0, 16.0, 2.0))

[6449,
 6454,
 6259,
 6452,
 4770,
 8131,
 6262,
 6447,
 6256,
 6455,
 4577,
 4771,
 7940,
 8133,
 4766,
 4774,
 8129,
 8136,
 4580,
 4767,
 4574,
 4773,
 7942,
 8130,
 7936,
 8135]

In [189]:
findNeighborTempsInit(Coordinate(16.0, 16.0, 2.0))

[300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0,
 300.0]

In [209]:
findNeighborTemperatures(Coordinate(16.0, 16.0, 2.0),1000)

[1033.56,
 1022.87,
 1033.13,
 1023.67,
 1021.78,
 1033.41,
 1038.37,
 1028.9,
 1027.67,
 1018.22,
 1026.75,
 1016.85,
 1038.08,
 1029.0,
 1027.3,
 1015.96,
 1038.41,
 1028.27,
 1032.27,
 1022.36,
 1020.91,
 1011.03,
 1043.07,
 1033.99,
 1032.93,
 1023.85]

In [37]:
p1 = Coordinate(0.5,1,0.5)
p2 = Coordinate(0.5,0.5,0.5)
timestep = 1000

In [38]:
selfTuple = p1.getXYZ()
otherTuple = p2.getXYZ()
    
if selfTuple in indices.keys() and otherTuple in indices.keys() and timestep>0:
    index_self = indices[selfTuple]
    index_other = indices[otherTuple]
    if index_other<index_self:
        temp_timestep = dataframe.loc[index_other].tolist()[timestep+4]
        print temp_timestep
    else:
        if timestep>1:
            temp_timestep = dataframe.loc[index_other].tolist()[timestep-1+4]
            print temp_timestep

print DEFAULT_TEMP

916.034
300


In [44]:
start = time.time()
for i in range(100):
    findAdjustedNeighborTemperatures(Coordinate(0.5,1,0.5),200)
stop = time.time()
print 'Time elapsed is',stop-start,'seconds'

Time elapsed is 24.3850610256 seconds


### <font color="red">Future Plans </font>
0. Do neighboring timestep -1 or timestep based on index 
1. Only do upto few layers 
2. Do PCA on the 26 neighbors 
3. Check for 2nd degree features 

### <font color="magenta">Smaller Simulation</font>

In [3]:
df = pd.read_csv('10-10-5-800/10-10-5-800.csv',header=None)

In [4]:
df_t = df.transpose()

In [5]:
df_t.head()

,0,1,2,3,4,5,6,7,8,9,...,924,925,926,927,928,929,930,931,932,933
0,0.5,1,0.5,0,300,4215.24,2646.37,1694.46,1317.48,1174.82,...,1231.96,1227.77,1223.72,1219.59,1215.94,1212.56,1209.23,1206.34,1203.66,1200.98
1,0,1.5,0.5,0,300,3560.53,2157.51,1479.99,1238.94,1116.14,...,1244.77,1240.44,1236.32,1232.23,1228.72,1225.52,1222.41,1219.74,1217.26,1214.79
2,0.5,1.5,0.5,0,300,3921.14,2510.44,1613.34,1314.85,1172.12,...,1254.4,1249.99,1245.8,1241.64,1238.07,1234.81,1231.64,1228.91,1226.38,1223.85
3,0.5,0,0.5,0,300,4105.97,2748.59,1673.09,1324.83,1179.21,...,1199.38,1195.53,1191.75,1187.8,1184.25,1180.89,1177.54,1174.63,1171.91,1169.19
4,0.5,0.5,0.5,0,300,4229.27,2694.35,1741.31,1326.04,1179.2,...,1208.52,1204.58,1200.72,1196.71,1193.13,1189.75,1186.4,1183.49,1180.78,1178.06


In [9]:
toolpath, state, endTime = parse_toolpath('10-10-5-800/','toolpath.crs',0.1)

In [19]:
indices_smallerSim,value_smallerSim = createObjectDictionaries(df_t)

The time for dictionary creation is 3.400483 seconds


In [22]:
df_t.to_csv('data_smallerSim.csv')

In [21]:
saveDict(indices_smallerSim,'indices_smallerSim')

indices_smallerSim saved successfully in .


In [23]:
timestep_Start = 1
timestep_Stop = 2

In [24]:
start = time.time()
neighborColumns,historicalColumns,dictList = [],[],[]

for historical in range(5):
    historicalColumns += ['Tminus'+str(historical+1)]

for neighbor in range(26):
    neighborColumns += ['T'+str(neighbor+1)+'_t-1']

columns = ['voxelLat','voxelLong','voxelVert','timestep','x_voxel','y_voxel','z_voxel','layerNum','time_creation','x_laser','y_laser','z_laser','x_distance','y_distance','z_distance','euclidean_distance_laser'] + historicalColumns+ neighborColumns+['T_self']

num_timesteps = len(dataframe.loc[0]-4)
num_voxels = len(dataframe)

for timestep in range(timestep_Start,timestep_Stop):#range(1,num_timesteps+1):
    for voxel in indices.keys():
        x,y,z = voxel

        voxelPosObject = Coordinate(x,y,z)
        T_self = getTemperature(voxelPosObject,timestep)
        if T_self == DEFAULT_TEMP:
            continue

        x_laser,y_laser,z_laser = tuple(toolpath[timestep-1])
        x_distance, y_distance, z_distance = abs(x-x_laser), abs(y-y_laser),abs(z-z_laser)
        layerNum = z/0.5

        laserPosObject = Coordinate(x_laser,y_laser,z_laser)

        distance_laser = voxelPosObject.distance(laserPosObject)

        dfIndex = indices[voxel]
        row = dataframe.loc[dfIndex].tolist()
        time_creation = float(row[3])
        time_current = float(timestep*0.1)
        time_elapsed = time_current - time_creation

        dictionary = {'voxelLat':'interior','voxelLong':'interior','voxelVert':'interior','timestep':timestep, 'x_voxel':x, 'y_voxel':y, 'z_voxel':z, 'layerNum':layerNum,
                     'time_creation':time_creation, 'x_laser':x_laser,'y_laser':y_laser,'z_laser':z_laser,
                     'x_distance':x_distance,'y_distance':y_distance,'z_distance':z_distance,
                      'distance_laser':distance_laser}


        for i in range(5):
            dictionary[historicalColumns[i]] = getTemperature(voxelPosObject,timestep-i-1)


        neighborTemps = findNeighborTemperatures(voxelPosObject,timestep)
        for i in range(len(neighborTemps)):
            key = 'T'+str(i+1)+'_t-1'
            if neighborTemps[i] == DEFAULT_TEMP:
                dictionary[key] = NON_EXISTING_TEMP
                if i==0 or i==1:
                    dictionary['voxelLat'] = 'missing'
                if i==3:
                    dictionary['voxelLong'] = 'missing'
                if i==5:
                    dictionary['voxelVert'] = 'missing'

            else:
                dictionary[key] = neighborTemps[i]


        dictionary['T_self'] = T_self

        dictList += [dictionary]

database =  pd.DataFrame(dictList,columns=columns)
stop = time.time()
print 'Database creation took',stop-start,'seconds'


NameError: name 'dataframe' is not defined

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_smallerSim.csv',header=None)

/Users/Arindam/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,2

In [3]:
df = pd.read_csv('data_smallerSim_1_2.csv')

IOError: File data_smallerSim_1_2.csv does not exist

In [11]:
neighborColumns,historicalColumns,dictList = [],[],[]

for historical in range(5):
    historicalColumns += ['Tminus'+str(historical+1)]

for neighbor in range(26):
    neighborColumns += ['T'+str(neighbor+1)+'_t-1']

columns = ['voxelLat','voxelLong','voxelVert','timestep','x_voxel','y_voxel','z_voxel','layerNum','time_creation','x_laser','y_laser','z_laser','x_distance','y_distance','z_distance','euclidean_distance_laser'] + historicalColumns+ neighborColumns+['T_self']


In [14]:
df = pd.DataFrame(loadNumpy('data_smallerSim_1_2'),columns=columns)

In [15]:
df.columns

Index([u'voxelLat', u'voxelLong', u'voxelVert', u'timestep', u'x_voxel',
       u'y_voxel', u'z_voxel', u'layerNum', u'time_creation', u'x_laser',
       u'y_laser', u'z_laser', u'x_distance', u'y_distance', u'z_distance',
       u'euclidean_distance_laser', u'Tminus1', u'Tminus2', u'Tminus3',
       u'Tminus4', u'Tminus5', u'T1_t-1', u'T2_t-1', u'T3_t-1', u'T4_t-1',
       u'T5_t-1', u'T6_t-1', u'T7_t-1', u'T8_t-1', u'T9_t-1', u'T10_t-1',
       u'T11_t-1', u'T12_t-1', u'T13_t-1', u'T14_t-1', u'T15_t-1', u'T16_t-1',
       u'T17_t-1', u'T18_t-1', u'T19_t-1', u'T20_t-1', u'T21_t-1', u'T22_t-1',
       u'T23_t-1', u'T24_t-1', u'T25_t-1', u'T26_t-1', u'T_self'],
      dtype='object')

In [16]:
df.head()

,voxelLat,voxelLong,voxelVert,timestep,x_voxel,y_voxel,z_voxel,layerNum,time_creation,x_laser,...,T18_t-1,T19_t-1,T20_t-1,T21_t-1,T22_t-1,T23_t-1,T24_t-1,T25_t-1,T26_t-1,T_self
0,interior,interior,interior,1,15.5,10.5,2.5,5,2.5,0,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
1,interior,interior,interior,1,16,16,2,4,2,0,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2,interior,interior,interior,1,16.5,15,2,4,2,0,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
3,interior,interior,interior,1,11.5,17.5,2.5,5,2.5,0,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
4,interior,interior,interior,1,8.5,12.5,4,8,4,0,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99


In [17]:
len(df)

11032

In [20]:
df['voxelLat'].value_counts(), df['voxelLong'].value_counts(), df['voxelVert'].value_counts()

(interior    10842
 missing       190
 Name: voxelLat, dtype: int64, interior    11031
 missing         1
 Name: voxelLong, dtype: int64, interior    11031
 missing         1
 Name: voxelVert, dtype: int64)